ITEM BASED COLLABORATIVE FILTERING - IBCF

In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from jproperties import Properties


In [2]:
class ProdReco:
    
    def evaluation(self,csv_file_path):
#Read the csv File
df = pd.read_csv('C:/Users/reraveen/Jupyter/Dataset/ORMB_Data_version.csv',na_values ='')
dfprocessed =df.groupby(['PERSON_ID', 'PRICEITEM_CD']).size().reset_index().rename(columns={0:'Score'})
train=dfprocessed.sample(frac=0.60,random_state=123) #random state is a seed value
test_Actual=dfprocessed.drop(train.index)
print(train.shape)
test = test_Actual.copy()
print(test.shape)
#Convert already bought product score to 0 for testing purpose
test.loc[:, 'Score'] =0
testDataset = pd.concat([train,test], ignore_index=True)
print(testDataset.shape)
TableCustItem =testDataset.pivot(index='PERSON_ID', columns='PRICEITEM_CD', values='Score')
TableCustItem =TableCustItem.replace(np.nan,0)
print(TableCustItem.shape)

df_crossTabedold = pd.crosstab(df.PERSON_ID, df.PRICEITEM_CD).reset_index().rename_axis('',axis='columns') 
df_crossTabedold.shape
TableCustItem =TableCustItem.merge(df_crossTabedold["PERSON_ID"], on='PERSON_ID', how='left')
df_crossTabed=TableCustItem
df_crossTabed1 = df_crossTabed.drop('PERSON_ID', 1)
df_crossTabed1.head()
df_crossTabed1.shape

magnitude = np.sqrt(np.square(df_crossTabed1).sum(axis=1))
df_crossTabed2 = df_crossTabed1.divide(magnitude, axis='index')
df_crossTabed2 =df_crossTabed2.replace(np.nan,0)

def calculate_similarity(data_items):
    """Calculate the column-wise cosine similarity for a sparse
    matrix. Return a new dataframe matrix with similarities.
    """
    data_sparse = sparse.csr_matrix(data_items)
    similarities = cosine_similarity(data_sparse.transpose())
    sim = pd.DataFrame(data=similarities, index= data_items.columns, columns= data_items.columns)
    return sim

data_matrix = calculate_similarity(df_crossTabed2)
data_matrix=data_matrix.replace(np.nan,0)
scoredf = data_matrix.dot(df_crossTabed2.T)
scoredf =scoredf.T
scoredf=scoredf.div(data_matrix.sum(axis=1))
scoredf=scoredf.replace(np.nan,0)
Bought_df=df_crossTabed2.where(df_crossTabed2==0,-999999)
recommd_df=Bought_df.where(Bought_df != 0,scoredf)
top_n = 7
df_final = pd.DataFrame({n: recommd_df.T[col].nlargest(top_n).index.tolist() 
                for n, col in enumerate(recommd_df.T)}).T

new_col =df_crossTabed['PERSON_ID']
df_final.insert(0,'PERSON_ID',new_col,True)
df_final

test_Actual_cross = pd.crosstab(test_Actual.PERSON_ID, test_Actual.PRICEITEM_CD).reset_index().rename_axis('',axis='columns') 
test_Actual_cross

testPersonId=test_Actual_cross['PERSON_ID']
test_Actual_cross=test_Actual_cross.drop(['PERSON_ID'],axis=1)



test_Actual_cross_df=test_Actual_cross.where(test_Actual_cross==0,-999999)
test_Actual_cross_df

Actual_BoughtList = (test_Actual_cross_df == -999999).sum(axis=1)
Actual_BoughtList

top_n = 7
Test_final = pd.DataFrame({n:test_Actual_cross.T[col].nlargest(top_n).index.tolist() 
                for n, col in enumerate(test_Actual_cross.T)}).T

Test_final.insert(0,'PERSON_ID',testPersonId,True)
MeredDf = Test_final.merge(df_final, on="PERSON_ID", how = 'inner')
MeredDf_Selected = MeredDf.loc[:,MeredDf.columns != 'PERSON_ID']
MeredDf['num_uniq'] = [len(set(v[pd.notna(v)].tolist())) for v in MeredDf_Selected.values]
MeredDf['num_uniq']=14-MeredDf['num_uniq']
MeredDf['precision']=MeredDf['num_uniq']/7
meanprecision = MeredDf['precision'].mean()
print("Precision :"+str(meanprecision))
rowCount =len(Test_final.index)
MeredDf['ActualBought']=Actual_BoughtList
actalBoughtCommon=[]
for j in range(0,rowCount):
    elements=[]
    actalCommon=0
    actualBought = Actual_BoughtList[j]
    eval_Value =7
    startindex = 7+1
    endInx =eval_Value*2+1
    elements.extend(Test_final.iloc[j,1:actualBought+1])
    elements.extend(MeredDf.iloc[j,startindex:endInx])
    a_set = set(elements)
    number_unique = len(a_set)
    actalCommon = (actualBought+eval_Value)-number_unique
    actalBoughtCommon.append(actalCommon)
actalBoughtCommon

MeredDf['ActualBoughtPredicted']=actalBoughtCommon
MeredDf['Recall']=MeredDf['ActualBoughtPredicted']/MeredDf['num_uniq']
meanrecall=MeredDf['Recall'].mean()
print("Recall :"+str(meanrecall))
mean_f1score = 2*(meanrecall*meanprecision)/(meanrecall+meanprecision)
print("F1-Score:"+str(mean_f1score))
        
    
    def recommend_building(self,csv_file_path):
        #Item-Item similarity function
        def calculate_similarity(data_items):
            """Calculate the column-wise cosine similarity for a sparse
            matrix. Return a new dataframe matrix with similarities.
            """
            data_sparse = sparse.csr_matrix(data_items)
            similarities = cosine_similarity(data_sparse.transpose())
            sim = pd.DataFrame(data=similarities, index= data_items.columns, columns= data_items.columns)
            return sim

        #Read the csv File
        df = pd.read_csv(csv_file_path,na_values ='')
        #A matrix repersentation of Person x Products
        df_crossTabed = pd.crosstab(df.PERSON_ID, df.PRICEITEM_CD).reset_index().rename_axis('',axis='columns') 
        df_crossTabed1 = df_crossTabed.drop('PERSON_ID', 1)
        #Normalizing the values
        magnitude = np.sqrt(np.square(df_crossTabed1).sum(axis=1))
        df_crossTabed2 = df_crossTabed1.divide(magnitude, axis='index')
        #Calling the similarity Function for Item-Item Matrix
        data_matrix = calculate_similarity(df_crossTabed2)
        #Changing the datatype of Person_Id to str
        df_crossTabed['PERSON_ID'] = df_crossTabed.PERSON_ID.astype(str)
        data_items = df_crossTabed.drop('PERSON_ID', 1)
        data_items2 = data_items.divide(magnitude, axis='index')
        score = data_matrix.dot(data_items2.T)
        score =score.T
        score=score.div(data_matrix.sum(axis=1))
        bought_df=data_items2.where(data_items2==0,-999999)
        recommd_df=bought_df.where(bought_df != 0,score)
        new_col =df_crossTabed['PERSON_ID']
        recommd_df.insert(0,'PERSON_ID',new_col,True)
        #recommd_df -have to save in a table/datastore
        recommd_df.to_json("recommendation1.JSON",orient="columns")
        


    #Recommendation Function
    def recommended_execution(self,customer_no,no_products):
        recommd_df = pd.read_json(r'recommendation1.JSON')
        recommd_df['PERSON_ID'] = recommd_df.PERSON_ID.astype(str)
        data_items = recommd_df.drop('PERSON_ID', 1)
        user_index = recommd_df[recommd_df.PERSON_ID == customer_no].index.tolist()[0]
        rating = data_items.loc[user_index] 
        recommend_products=rating.nlargest(no_products)
        return  recommend_products

#Initialize the object for class prodReco
product_reco_object = ProdReco()
#get the configuration for the input dataset
configs = Properties()
with open('config.properties', 'rb') as config_file:
    configs.load(config_file)
csv_file_path = configs["filePath"].data
product_reco_object.recommend_building(csv_file_path)
product_reco_object.evaluation(csv_file_path)
#Recommendation for given CustomerNo=72231957, and the number of products to recommend as Nproducts=5
recommed_products= product_reco_object.recommended_execution('72231957',5)
print(recommed_products)

SIV123                            0.140681
DM_P2                             0.117106
AB_CCARDS                         0.076253
AB_CWN                            0.052254
AB_AMF                            0.048927
Name: 0, dtype: float64
